In [37]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.helper_funcs import candles_to_df
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.simulate import run_df_backtest, or_backtest
from datetime import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import numpy as np
from numpy.core.multiarray import array as array
import plotly.graph_objects as go

from logging import getLogger
from typing import NamedTuple

from quantfreedom.helper_funcs import cart_product
from quantfreedom.indicators.tv_indicators import sma_tv
from quantfreedom.enums import CandleBodyType
from quantfreedom.strategies.strategy import Strategy

logger = getLogger("info")

class IndicatorSettingsArrays(NamedTuple):
    sma_fast_length: np.array
    sma_slow_length: np.array
    
class SMACrossing(Strategy):
    def __init__(self,
                long_short: str,
                sma_fast_length: np.array,
                sma_slow_length: np.array,
                ) -> None:
        self.long_short = long_short
        
        # Will return the cartisian product of the sma_fast_length and sma_slow_length
        cart_arrays = cart_product(
            IndicatorSettingsArrays(
                sma_fast_length = sma_fast_length,
                sma_slow_length = sma_slow_length,
                                   
            )
        )
        
        sma_fast_length = cart_arrays[0].astype(np.int_)
        sma_slow_length = cart_arrays[1].astype(np.int_)
        
        # We don't want any of the moving averages to be less than the fast length
        # The fast length acutally has to be slower than the slow length
        sma_bools = sma_fast_length < sma_slow_length
        
        self.indicator_settings_arrays: IndicatorSettingsArrays = IndicatorSettingsArrays(
            sma_fast_length = sma_fast_length[sma_bools],
            sma_slow_length = sma_slow_length[sma_bools]
        )
        
        if long_short == "long":
            self.set_entries_exits_array = self.long_set_entries_exits_array
            self.log_indicator_settings = self.long_log_indicator_settings
            self.entry_message = self.long_entry_message
        else:
            self.set_entries_exits_array = self.short_set_entries_exits_array
            self.log_indicator_settings = self.short_log_indicator_settings
            self.entry_message = self.short_entry_message
            
            
            
            
            
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
    ################################             Long             #########################################
    ################################             Long             #########################################
    ################################             Long             #########################################
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
            
        
    def long_set_entries_exits_array(self,
                                    candles: np.array,
                                    ind_set_index: int):
        try:
            closes = candles[:, CandleBodyType.Close]
            
            # sma fast
            # Grab the first sma_fast_length number of closes
            self.sma_fast_length = self.indicator_settings_arrays.sma_fast_length[ind_set_index]
            sma_fast = sma_tv(
                source = closes,
                length = self.sma_fast_length
            )
            
            self.sma_fast = np.around(sma_fast, 2)
            
            self.prev_sma_fast = np.roll(self.sma_fast, 1)
            self.prev_sma_fast[0] = np.nan
            logger.info(f'Created sma_fast and sma_fast_length = {self.sma_fast_length}')
            
            # same slow
            # Grab the first sma_slow_length number of closes
            self.sma_slow_length = self.indicator_settings_arrays.sma_slow_length[ind_set_index]
            sma_slow = sma_tv(
                source = closes,
                length = self.sma_slow_length
            )
            
            self.sma_slow = np.around(sma_slow, 2)
            
            self.prev_sma_slow = np.roll(self.sma_slow, 1)
            self.prev_sma_slow[0] = np.nan
            logger.info(f'Created sma_slow and sma_slow_length = {self.sma_slow_length}')
            
            # Entries
            self.entries = (self.prev_sma_fast < self.prev_sma_slow) & (self.sma_fast > self.sma_slow)
            self.cross_above_signal = np.where(self.entries, sma_fast, np.nan)
            logger.debug(f'Created entries and cross above signals')
            
            # Exits
            exits = (self.prev_sma_fast > self.prev_sma_slow) & (self.sma_fast < self.sma_slow)
            self.cross_below_signal = np.where(exits, sma_fast, np.nan)
            self.exit_rprices = np.roll(np.where(exits, closes, np.nan), 1)
            logger.debug(f'Created exit prices and cross below signals')

        except Exception as e:
            logger.warning(f'Error in long_set_entries_exits_array -> {e}')
            raise Exception(f'Exception long_set_entries_exits_array -> {e}')
            
            
    def long_log_indicator_settings(
        self,
        ind_set_index: int
    ):
        logger.info(
            f'Indicator Settings Index = {ind_set_index}\
            \nsma_fast_length = {self.sma_fast_length}\
            \nsma_slow_length = {self.sma_slow_length}'
        )
        
    def long_entry_message(
        self,
        bar_index: int
    ):
        logger.info('\n\n')
        logger.info(f'ENTRY, prev_sma_fast < prev_sma_slow & sma_fast > sma_slow')
        logger.info(f'{self.prev_sma_fast[bar_index]} < {self.prev_sma_slow[bar_index]} & {self.sma_fast[bar_index]} > {self.sma_slow[bar_index]}')
        
        
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
    ################################             Short            #########################################
    ################################             Short            #########################################
    ################################             Short            #########################################
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
        
        
        
        
        
    def short_set_entries_exits_array(self,
                                      candles: np.array,
                                      ind_set_index: int):
        try:
            closes = candles[:, CandleBodyType.Close]
            
            # sma fast
            # Grab the first sma_fast_length number of closes
            self.sma_fast_length = self.indicator_settings_arrays.sma_fast_length[ind_set_index]
            sma_fast = sma_tv(
                source = closes,
                length = self.sma_fast_length
            )
            
            self.sma_fast = np.around(sma_fast, 2)
            
            self.prev_sma_fast = np.roll(self.sma_fast, 1)
            self.prev_sma_fast[0] = np.nan
            logger.info(f'Created sma_fast and sma_fast_length = {self.sma_fast_length}')
            
            # same slow
            # Grab the first sma_slow_length number of closes
            self.sma_slow_length = self.indicator_settings_arrays.sma_slow_length[ind_set_index]
            sma_slow = sma_tv(
                source = closes,
                length = self.sma_slow_length
            )
            
            self.sma_slow = np.around(sma_slow, 2)
            
            self.prev_sma_slow = np.roll(self.sma_slow, 1)
            self.prev_sma_slow[0] = np.nan
            logger.info(f'Created sma_slow and sma_slow_length = {self.sma_slow_length}')
            
            # Entries
            self.entries = (self.prev_sma_fast > self.prev_sma_slow) & (self.sma_fast < self.sma_slow)
            self.cross_above_signal = np.where(self.entries, sma_fast, np.nan)
            logger.debug(f'Created entries and cross above signals')

            # Exits
            exits = (self.prev_sma_fast < self.prev_sma_slow) & (self.sma_fast > self.sma_slow)
            self.cross_below_signal = np.where(exits, sma_fast, np.nan)
            self.exit_rprices = np.roll(np.where(exits, closes, np.nan), 1)
            logger.debug(f'Created exit prices and cross below signals')
            
        except Exception as e:
            logger.warning(f'Error in short_set_entries_exits_array -> {e}')
            raise Exception(f'Exception short_set_entries_exits_array -> {e}')
                
        
    def short_log_indicator_settings(
        self,
        ind_set_index: int
    ):
        logger.info(
            f'Indicator Settings Index = {ind_set_index}\
            \nsma_fast_length = {self.sma_fast_length}\
            \nsma_slow_length = {self.sma_slow_length}'
        )
        
    def short_entry_message(
        self,
        bar_index: int
        ):
        logger.info('\n\n')
        logger.info(f'ENTRY, prev_sma_fast > prev_sma_slow & sma_fast < sma_slow')
        logger.info(f'{self.prev_sma_fast[bar_index]} > {self.prev_sma_slow[bar_index]} & {self.sma_fast[bar_index]} < {self.sma_slow[bar_index]}')
        
        
        
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
    ################################             Live             #########################################
    ################################             Live             #########################################
    ################################             Live             #########################################
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
        
        
    # Have to implement this once, I get actual strategies and learn more about the programming
    def live_set_indicator(self,
                           ind_set_index: int
    ):
        pass

    def live_evaluate(
        self,
        candles: np.array,
    ):
        pass
    
    
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################
    ################################             Plot             #########################################
    ################################             Plot             #########################################
    ################################             Plot             #########################################
    #######################################################################################################
    #######################################################################################################
    #######################################################################################################


    def plot_signals(self,
                     candles: np.array):
        
        timestamp = candles[:, CandleBodyType.Timestamp]
        datetimes = timestamp.astype('datetime64[ms]')
        open = candles[:, CandleBodyType.Open]
        high = candles[:, CandleBodyType.High]
        low = candles[:, CandleBodyType.Low]
        close = candles[:, CandleBodyType.Close]
        
        fig = go.Figure(
            data = [
                go.Candlestick(
                    x = datetimes,
                    open = open,
                    high = high,
                    low = low,
                    close = close,
                    name = 'Candles'
                ),
                go.Scatter(
                    x = datetimes,
                    y = self.sma_fast,
                    name = 'SMA Fast',
                    color = 'lightblue'
                ),
                go.Scatter(
                    x = datetimes,
                    y = self.sma_slow,
                    name = 'SMA Slow',
                    line_color = 'yellow'
                ),
                go.Scatter(
                    x = datetimes,
                    y = self.cross_above_signal,
                    mode = 'markers',
                    name = 'Entries',
                    marker = dict(
                        size = 12,
                        symbol = 'triangle-up',
                        color = 'green',
                        line = dict(
                            width = 1,
                            color = 'DarkSlateGrey'
                        )
                    )
                ),
                go.Scatter(
                    x = datetimes,
                    y = self.cross_below_signal,
                    mode = 'markers',
                    name = 'Exits',
                    marker = dict(
                        size = 12,
                        symbol = 'triangle-down',
                        color = 'red',
                        line = dict(
                            width = 1,
                            color = 'DarkSlateGrey'
                        )
                    )
                )
            ]
        )
        fig.update_layout(
            height = 800,
            xaxis_rangeslider_visible = False,
            title = dict(
                x = 0.5,
                text = 'Signals',
                xanchor = 'center',
                font = dict(
                    size = 40
                )
            )
        )
        fig.show()
        
    # For the live version
    
    def get_strategy_plot_filename(
        self,
        candles: np.array):
        
        pass

In [39]:
mufex_main = Mufex(
    api_key = MufexKeys['api_key'],
    secret_key = MufexKeys['secret_key'],
    use_test_net=False
)

In [40]:
candles = mufex_main.get_candles(
    symbol = 'BTCUSDT',
    timeframe = '30m',
    candles_to_dl = 3000,
    # since_datetime = datetime(2021, 1, 1)
    # until_datetime = datetime(2021, 1, 1)
)

In [41]:
# Setting the exchange settings twice gets an error when setting the leverage mode
mufex_main.set_exchange_settings(
    symbol = 'BTCUSDT',
    position_mode= PositionModeType.HedgeMode,
    leverage_mode= LeverageModeType.Isolated,
)


Exception: Mufex: set_position_mode= permission denied -> 

In [ ]:
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct = np.array([12]),
    max_trades = np.array([0]),
    risk_account_pct_size = np.array([3]),
    risk_reward = np.array([0]),
    sl_based_on_add_pct = np.array([0.05, 0.15]),
    sl_based_on_lookback = np.array([30]),
    sl_bcb_type = np.array([CandleBodyType.Low]),
    sl_to_be_cb_type = np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct = np.array([0]),
    trail_sl_bcb_type = np.array([CandleBodyType.Low]),
    trail_sl_by_pct = np.array([1, 2]),
    trail_sl_when_pct = np.array([1, 3])
)
static_os = StaticOrderSettings(
    increase_position_type = IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type = LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb = 'min',
    sl_strategy_type = StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool = False,
    starting_bar = 50,
    starting_equity = 1000.0,
    static_leverage = None,
    tp_fee_type = 'market',
    tp_strategy_type = TakeProfitStrategyType.Provided,
    trail_sl_bool = True,
    z_or_e_type = None,
)

strategy = SMACrossing(
    long_short = "long",
    sma_fast_length = np.arange(20, 61, 20),
    sma_slow_length = np.arange(30, 91, 30)
)

In [ ]:
backtest_results = run_df_backtest(
    backtest_settings = backtest_settings,
    candles = candles,
    dos_arrays = dos_arrays,
    exchange_settings = mufex_main.exchange_settings,
    static_os = static_os,
    strategy = strategy,
    # logger_bool = False
)

AttributeError: 'NoneType' object has no attribute 'market_fee_pct'